In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Get the OpenAI API key from the .env file
load_dotenv('.env', override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = openai_api_key)

In [3]:
def get_llm_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant.",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )
    response = completion.choices[0].message.content
    return response

In [4]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 14.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 121.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pymupdf


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [9]:
%pip install pytesseract pillow


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
!pip install pdfplumber openai python-dotenv ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 125.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [11]:
import pdfplumber
from PyPDF2 import PdfReader

import ipywidgets as widgets
from IPython.display import display

In [12]:
# Step 2: File upload widget for Jupyter
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload_widget)


FileUpload(value=(), accept='.pdf', description='Upload')

In [13]:
# Step 3: Save uploaded PDF file locally
def save_uploaded_file(upload_widget):
    if upload_widget.value and len(upload_widget.value) > 0:
        file_info = upload_widget.value[0]
        file_name = file_info['name']
        content = file_info['content']
        with open(file_name, "wb") as f:
            f.write(content)
        return file_name
    return None

In [14]:
# After uploading, run this cell manually
file_path = save_uploaded_file(upload_widget)
print("Saved file:", file_path)


Saved file: ocr test 2.pdf


In [15]:
# Step 4: Extract raw text from PDF using pdfplumber
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

In [17]:
# Step 5: Extract form fields (includes radio buttons / checkboxes) using PyPDF2
def extract_pdf_form_fields(file_path):
    reader = PdfReader(file_path)
    fields = reader.get_fields()
    result = {}
    if fields:
        for key, val in fields.items():
            if isinstance(val, dict):
                value = val.get('/V') or val.get('/AS')
                if value and hasattr(value, 'original_bytes'):
                    value = value.original_bytes.decode('utf-8')
                if isinstance(value, str) and value.startswith('/'):
                    value = value[1:]
                result[key] = value
            else:
                result[key] = val
    return result

In [18]:
# Step 6: OpenAI LLM call for JSON extraction from raw text
def get_llm_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant.", 
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.0, # change this to a value between 0 and 2
    )
    response = completion.choices[0].message.content
    return response

In [19]:
# Step 7: Convert PDF text to JSON using LLM with given fields
def parse_pdf_to_json(file_path):
    form_fields = extract_pdf_form_fields(file_path)
    raw_text = extract_text_from_pdf(file_path)

    # Convert form fields dictionary into a string representation for the prompt
    form_fields_str = "\n".join([f"{k}: {v}" for k,v in form_fields.items()])

    prompt = f"""
You are a document parser. Your task is to convert the provided PDF text and form data
into structured JSON.

Rules:

1. Include all fields in the output JSON, even if their value is empty or null.
2. For checkboxes or radio buttons:
   -Include only the items that are selected (marked with '✔' or similar).
   - If no items in a checkbox/radio group are selected, set the field to null.
   - Do NOT output boolean true/false for unselected items.
   - Extract only the items that are marked as selected, indicated by a leading '✔' or similar mark.
    - Ignore items without any selection mark.
    - If no items are selected in a checklist, set it to null.

3. Split full names into 'FirstName' and 'LastName' if possible.
4. Group logically related fields together.
5. Output valid JSON only. No explanation, no extra text.
6.Extract data if it is split into two coloumns and/or numbered columns data

Here are the filled form fields (checkboxes, radios, etc.):

{form_fields_str}

Here is the extracted text from the PDF:

{raw_text}
"""

    json_output = get_llm_response(prompt)
    return json_output


In [21]:
# Step 8: Run everything and show combined result
if file_path:
    # Extract raw text
    raw_text = extract_text_from_pdf(file_path)
    print(raw_text)
    
    # Extract PDF interactive form fields
    form_fields = extract_pdf_form_fields(file_path)
    print(form_fields)

    # Convert raw text to JSON with LLM
    extracted_json = pdf_text_to_json(raw_text)
    print("----- Extracted JSON from Text -----")
    print(extracted_json)
else:
    print("Please upload a PDF file above and rerun this cell.")

(cid:144)(cid:133)(cid:6)(cid:129)(cid:143)(cid:143)(cid:135)(cid:130)(cid:135))(cid:140)(cid:6)(cid:137)(cid:132)(cid:134)+(cid:135)((cid:132)(cid:6)(cid:129)(cid:143)(cid:6)((cid:138)(cid:132)(cid:6)(cid:145)(cid:133)(cid:135)((cid:132)(cid:136)(cid:6)’()((cid:132)+(cid:6)(cid:146)(cid:129)(cid:147)(cid:132)(cid:148)(cid:133)(cid:131)(cid:132)(cid:133)((cid:149)(cid:150)(cid:151)(cid:150)(cid:152)(cid:153)(cid:6)(cid:154)(cid:155)(cid:156)(cid:6)(cid:157)(cid:155)(cid:158)(cid:6)(cid:159)(cid:160)(cid:155)(cid:156)
(cid:0)(cid:2)(cid:3)(cid:4)(cid:5)(cid:2)(cid:6)(cid:7)(cid:8)(cid:6)(cid:9)(cid:10)(cid:11)(cid:2)(cid:12)(cid:13)(cid:3)(cid:14)(cid:6)(cid:5)(cid:10)(cid:11)(cid:6)(cid:15)(cid:4)(cid:16)(cid:2)(cid:3)(cid:17)(cid:13)(cid:14)
(cid:18)(cid:19)(cid:15)(cid:19)(cid:6)(cid:20)(cid:4)(cid:21)(cid:5)(cid:3)(cid:13)(cid:22)(cid:4)(cid:10)(cid:13)(cid:6)(cid:7)(cid:8)(cid:6)(cid:23)(cid:7)(cid:22)(cid:22)(cid:4)(cid:3)(cid:16)(cid:4)
,-./01213(cid:6)4(cid:6)5//6-713(cid:6)89,-

NameError: name 'pdf_text_to_json' is not defined